# 1. Train  and test LeNet using caffe
Jupyter notebook을 사용하여 Linux Shell 명령어로 caffe를 사용하는 방법에 대해서 배웁니다.

본 예제는 크게 두 Part로 나누어져 있습니다.
1. Caffe 명령어를 통해 모델을 학습하는 방법
2. 학습된 모델의 test data에 대한 Classification 성능을 측정하는 방법

### 제공해드린 Docker 이미지 상에 이미 Caffe가 ./caffe에 설치되어 있으며, 이를 기준으로 본 tutorial이 작성되어 있습니다. 

### 0. Basic Usage

* Jupyter Notebook에서 Linux Shell 명령어를 사용하기 위해서는 명령앞에 !를 붙여서 실행하면 됩니다.

In [ ]:
!pwd

In [ ]:
!ls .

## 1. Training LeNet 5

<img src=./lenet/lenet5.png>

### 1-1. Convert MNIST dataset

본 예제에서 사용할 데이터셋은 MNIST로 0~9까지의 숫자 이미지와 이미지에 해당하는 Label을 포함합니다.
MNIST 데이터셋에 대해 자세한 내용은 http://yann.lecun.com/exdb/mnist 를 참고해 주세요.

Raw MNIST dataset은 제공해드린 docker 이미지상에 이미 다운로드 되어 있으며, 해당 위치는 ./caffe/data/mnist 입니다.
하지만 Raw MNIST dataset의 형식은 caffe에서 지원되지 않기에 caffe에서 가장 많이 쓰는 LMDB형식으로 변환되어야 합니다.
변환에 필요한 코드는 이미 작성되어 있으며 ./lenet/create_mnist.sh 을 실행하여 변환합니다.

스크립트 실행시 발생하는 libdc1394 error는 docker의 driver상의 오류로 무시하셔도 됩니다

In [ ]:
!sh ./lenet/create_mnist.sh

변환이 완료된 후, !ls ./lenet 명령어를 통해 mnist_test_lmdb와 mnist_train_lmdb가 정상적으로 생성되었음을 확인하실 수 있습니다.

In [ ]:
!ls ./lenet/*_lmdb

## 1-2. Define model architecture

LeNet5의 모델은 ./lenet/lenet_train_test.prototxt에 정의되어 있습니다. 

cat 명령어는 Linux에서 text파일을 모니터에 출력해주는 함수로 prototxt 파일을 읽어올 수 있습니다.
http://caffe.berkeleyvision.org/tutorial/layers.html 를 참고하여 각 layer가 요구하는 parameter를 참고하세요.

prototxt는 jupyter notebook 상에서 해당 파일 ./lenet/lenet_train_test.prototxt 을 클릭하는 것으로 수정하실 수 있습니다.

In [ ]:
!cat ./lenet/lenet_train_test.prototxt

## 1-3. Define solver

Network를 학습하기 위해서 Solver를 prototxt형식으로 정의합니다. 

Solver를 정의하는 다양한 Parameter는 http://caffe.berkeleyvision.org/tutorial/solver.html 를 참고해주세요.

Solver hyperparameters
- Solver: SGD (Stochastic gradient descent)
- Base Learning Rate: 0.01 (Staring learning rate)
- Momemtum: 0.9 (SGD parameter)
- weight_decay: 0.005
- lr_policy: inv  
- Gamma: 0.0001
- Power: 0.75
- solver_mode: CPU  

Display param
- display: 100 (training 100 batch 마다 Loss 출력)
- test_iter: 100 (validation data시 100개의 batch를 테스트)
- test_interval: 500 (training 500 batch 마다 validation data test)
- max_iter: 1000 (1000 batch training 도달시 종료)

Snapshot param
- snapshot: 100 (100 batch training 마다 모델 저장)
- snapshot_prefix: ./lenet/lenet (모델 저장 위치와 이름에 대한 prefix)

위의 내용을 caffe의 prototxt 형식으로 나타내면 아래와 같습니다.

In [ ]:
!cat ./lenet/lenet_solver.prototxt

## 1-4. Train LeNet5

빌드된 Caffe의 실행파일은 ./caffe/build/tools/caffe에 존재합니다.

caffe 실행파일의 사용법(commands & arguments)을 보기 위해서 아래의 명령어를 실행합니다.

In [ ]:
!./caffe/build/tools/caffe

가장 간단하게는 ./caffe/build/tools/caffe train --solver=/path/to/solver/file.prototxt로 정의된 모델을 정의된 solver로 학습시킬 수 있습니다.

## 컴퓨터 사양에 따라 소요시간이 증가할 수 있습니다. 중단을 원하시면 Jupyter notebook 상단의 Kernel 탭 클릭 후 Interrupt를 클릭해주세요.
#### Shell 상에서 중단 (Jupyter notebook 상의 kernel interrupt)이 될 경우 중단 시점에서의 snapshot이 자동저장됩니다.

### Jupyter notebook 상의 ln [\*] 표시는 현재 처리중임을 의미하며, 완료가 되면 \* 기호는 숫자로 변경됩니다.

In [ ]:
!./caffe/build/tools/caffe train --solver=./lenet/lenet_solver.prototxt

Caffe에서는 예기치 않은 이유로 학습이 중단되었을 경우 저장된 지점(.solverstate 파일)이 있다면 이어서 학습이 가능합니다. (Snapshot resume 기능)

100번까지 학습된 Solver state가 있다면 아래의 명령어를 통해 iteration 100번부터 이어서 학습이 가능합니다.

In [ ]:
!./caffe/build/tools/caffe train --solver=./lenet/lenet_solver.prototxt --snapshot=./lenet/lenet_iter_100.solverstate

본 과정이 모두 완료되면 10000 minibatch의 학습 후 Test dataset에서 약 99.1%의 인식 성능을 갖는 모델(./lenet/lenet_iter_10000.caffemodel)을 얻을 수 있습니다.


In [ ]:
!ls ./lenet

## 2. Testing LeNet 5

학습된 모델으로부터 Test set에 대한 성능을 측정해보겠습니다.
MNIST dataset의 test sample 개수가 10000개이며, lenet_train_test.prototxt의 test batchsize가 100개로 설정되어 있으므로 총 100번의 Iteration을 수행하면 됩니다.

즉 ./caffe/build/tools/caffe를 실행시킬때,
- 학습된 모델을 사용하여 inference를 하기에 __test__의 인자를 넣어야 하며,
- 학습된 모델의 definition prototxt __./lenet/lenet_train_test.prototxt__
- 학습된 모델의 weights __./lenet/lenet_iter_1000.caffemodel__
을 차례로 적용하면 됩니다.

In [ ]:
!./caffe/build/tools/caffe test --model=./lenet/lenet_train_test.prototxt --weights=./lenet/lenet_iter_1000.caffemodel --iterations=100

위의 코드의 수행 결과 가장 아래쪽에 100개의 minibatch상에서의 평균 성능이 나오며, 정상적으로 본 tutorial을 수행한다면 99.1%의 accuracy를 보실 수 있습니다.